In [ ]:
!pip install bnlearn

In [ ]:
!pip install d3blocks

In [ ]:
import warnings

# To ignore all warnings
warnings.filterwarnings("ignore")

# BnLearn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import bnlearn as bn
import networkx as nx
from tabulate import tabulate

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Replace 'your_file.xlsx' with the actual name of your Excel file
file_path = 'after_filtering_data.csv'

# Read the Excel file into a pandas DataFrame or csv file
df1 = pd.read_csv(file_path)

# Display the DataFrame
df1.head()

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

inertias = []

# Compute the inertia (WCSS) for each number of clusters from 1 to 10
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(df1)
    inertias.append(kmeans.inertia_)  # Inertia is the same as WCSS

# Plot the elbow method graph
plt.plot(range(1, 11), inertias, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia (WCSS)')
plt.show()

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

In [ ]:
discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
df_discretized = pd.DataFrame(discretizer.fit_transform(df1), columns=df1.columns)

print(df_discretized)

df_discretized.describe()

In [ ]:
# Structure learning
model = bn.structure_learning.fit(df_discretized, methodtype='hillclimbsearch', scoretype='bic')

# Test edges significance and remove.
model = bn.independence_test(model, df_discretized, test="chi_square", alpha=0.05, prune=True)

# Make plot
G = bn.plot(model, interactive=False)

# Make plot
# Set the figure size
plt.figure(figsize=(20, 10))  # Adjust the width and height as needed

# Show the plot
plt.show()

# Tabulate the results of the independence test
# Examine the output of the chi-square test. All P values are significant. Nothing is removed.
results1 = tabulate(model['independence_test'], tablefmt="grid", headers="keys")
print(results1)

# CausalNex

In [ ]:
!pip install causalnex

In [ ]:
from causalnex.structure.notears import from_pandas
from causalnex.network import BayesianNetwork
import networkx as nx
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
# Structure learning
sm = from_pandas(df1)
# Print information about the structure model
print(sm.edges(data=True))

# Thresholding
sm.remove_edges_below_threshold(0.8)

# Use positions from Bnlearn
pos = nx.spring_layout(sm)
# Make plot
plt.figure(figsize=(15,10));
edge_width = [ d['weight']*0.3 for (u,v,d) in sm.edges(data=True)]
nx.draw_networkx(sm, node_size=200, arrowsize=20, alpha=0.6, edge_color='b', width=edge_width)


In [ ]:
# Extract edge information from the structure model
edge_info = [(u, v, d) for u, v, d in sm.edges(data=True)]

# Create a DataFrame from the edge information
df_edge_info = pd.DataFrame(edge_info, columns=['Source', 'Target', 'Attributes'])

# Display the DataFrame
df_edge_info

# LiNGAM

In [ ]:
!pip install causal-learn

In [ ]:
! pip install lingam

In [ ]:
import numpy as np
import pandas as pd
import graphviz
from graphviz import Source
import networkx as nx
import pydotplus
from causallearn.search.FCMBased import lingam
from causallearn.search.FCMBased.lingam.utils import make_dot
import matplotlib.pyplot as plt

np.set_printoptions(precision=3, suppress=True)

In [ ]:
# Assuming df1 is your DataFrame with data
model_ica = lingam.ICALiNGAM(random_state=42)
model_ica.fit(df1)

# Create labels for each column in df1
labels = [f'{col}' for col in df1.columns]

# Generate the plot using make_dot
make_dot(model_ica.adjacency_matrix_, labels=labels)

In [ ]:
# Initialize an empty DataFrame to store all edge information
all_edges_df = pd.DataFrame(columns=['From', 'To'])

# Convert adjacency matrix into DataFrame for better visualization
adjacency_df = pd.DataFrame(model_ica.adjacency_matrix_, index=labels, columns=labels)

# Initialize a list to store edge information for this iteration
edges_list = []

# Iterate over the adjacency matrix and add edges to edges_df
for i in range(adjacency_df.shape[0]):
    for j in range(adjacency_df.shape[1]):
        adjacency_value = adjacency_df.iloc[i, j]  # Get the value in the adjacency matrix
        if adjacency_value != 0:
            connection_strength = adjacency_value  # The connection strength is the same as the adjacency value
            edges_list.append({'From': labels[j], 'To': labels[i], 'Connection Strength': connection_strength})

# Convert edges_list to a DataFrame and append it to all_edges_df
edges_df = pd.DataFrame(edges_list)
all_edges_df = pd.concat([all_edges_df, edges_df], ignore_index=True)

# Replace 'path_to_save.csv' with your desired file path and name
all_edges_df